In [1]:
import psycopg2
import tweepy
import json
import csv

# Tweepy API doc here: http://pythonhosted.org/tweepy/html/api.html
# psycopg2 API doc here: http://initd.org/psycopg/docs/

In [2]:
credentials = {}
credentials['api_key'] = 'n2dTtVbZuJwji0auruBGVTBwT'
credentials['api_secret_key'] = 'U1Er7hNUFtOwmPIz6BCr6w1B3o9GlKhg6loffJVIK0cFAFdkYl'
credentials['access_token'] = '987093510116532231-tiHjc16Ws3vcMZpNUt0kuL7xj8iLSy1'
credentials['access_token_secret'] = 'ykZmkTz2TIeSajyWDKXvSSxAiFZvMFdD9ePUe0zn5fsna'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

# Load Twitter API secrets from an external JSON file
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

# Twitter initialization
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
# Postgresql initialization
from configparser import ConfigParser
def config(filename='database.ini', section='postgresql'): #C:/temp/pract_1/database.ini
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return db

def connect():
 """ Connect to the PostgreSQL database server """
 conn = None
 try:
     # read connection parameters
     params = config()
     # connect to the PostgreSQL server
     print('Connecting to the PostgreSQL database...')
     conn = psycopg2.connect(**params)
     # create a cursor
     cur = conn.cursor()

    # execute a statement
     print('PostgreSQL database version:')
     cur.execute('SELECT version()')
     # display the PostgreSQL database server version
     db_version = cur.fetchone()
     print(db_version)

     # close the communication with the PostgreSQL
     cur.close()
 except (Exception, psycopg2.DatabaseError) as error:
     print(error)
 finally:
     if conn is not None:
         conn.close()
     print('Database connection closed.')

In [4]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        DROP TABLE IF EXISTS users CASCADE;
        """,
        """
        CREATE TABLE users (
            user_id VARCHAR(30) PRIMARY KEY,
            user_name VARCHAR(30),
            followers_count VARCHAR(30),
            friends_count VARCHAR(30)
        );
        """,

        """
        DROP TABLE IF EXISTS tweets CASCADE;
        """,
        """
        CREATE TABLE tweets (
            tweet_id VARCHAR(30) PRIMARY KEY,
            user_id VARCHAR(30),
            FOREIGN KEY (user_id) REFERENCES users (user_id) ON DELETE CASCADE,
            created_at VARCHAR(30),
            text VARCHAR(300),
            favorite_count VARCHAR(300), 
            in_reply_to_screen_name VARCHAR(30),
            retweeted VARCHAR(5)
        );
        """
    )
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [5]:
def insert_user(user_id, user_name, followers_count, friends_count):
    sql = """INSERT INTO users(user_id, user_name, followers_count, friends_count)
             VALUES(%s, %s, %s, %s) RETURNING user_id;"""
    conn = None
    id = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute(sql, (user_id, user_name, followers_count, friends_count))
        # get the generated id back
        id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return id

In [6]:
def insert_tweet(tweet_id, user_id, created_at, text, favorite_count, in_reply_to_screen_name, retweeted):
    sql = """INSERT INTO tweets(tweet_id, user_id, created_at, text, favorite_count, in_reply_to_screen_name, retweeted)
             VALUES(%s, %s, %s, %s, %s, %s, %s) RETURNING tweet_id;"""
    conn = None
    id = None
    try:
        # read database configuration
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute(sql, (tweet_id, user_id, created_at, text, favorite_count, in_reply_to_screen_name, retweeted))
        # get the generated id back
        id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
 
    return id

In [7]:
# Helper function to get all tweets for a specified user
# NOTE:  This method only allows access to the most recent 3240 tweets
# Source: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_name):
    # initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    # save most recent tweets
    alltweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        # save most recent tweets
        alltweets.extend(new_tweets)
        
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print("...%s tweets downloaded so far" % (len(alltweets)))
            
    #write into postgres
    insert_user(alltweets[0].user.id_str, alltweets[0].user.screen_name, alltweets[0].user.followers_count, alltweets[0].user.friends_count)
    for tweet in alltweets:
        insert_tweet(tweet.id_str, tweet.user.id_str, tweet.created_at, tweet.text, tweet.favorite_count, tweet.in_reply_to_screen_name, tweet.retweeted)

In [8]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

In [9]:
def simple_query_1():
    """ get the tweets between two dates"""
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT (tweets.created_at, tweets.text) FROM tweets WHERE CAST(created_at as date) BETWEEN '20200101' and '20200201'
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [10]:
def simple_query_2():
    """ get the users who have 40 or more followers"""
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT (users.followers_count, users.screen_name) FROM users WHERE users.followers_count>=CAST(40 AS varchar(1))
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [11]:
def join_query_1():
    """ get the users and tweets in which users have more than 30 followers and there are more than 5 retweets """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT (users.user_name, tweets.text) FROM tweets INNER JOIN users ON users.followers_count>=CAST(30 AS varchar(1)), tweets.favorite_count>=CAST(5 AS varchar(1))
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [12]:
def aggregate_query_1():
    """ get the user that has more followers """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT users.user_name FROM users WHERE users.followers_count= (SELECT min(users.followers_count) FROM users)
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [13]:
def aggregate_query_2():
    """ order the users by followers in crecent order """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT users.user_name, min(users.friends_count) FROM users GROUP BY users.user_name
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [14]:
def search_query_1():
    """ search for the word iPhone """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""
            SELECT tweets.text FROM tweets WHERE tweets.text LIKE '%iPhone%'
        """)
        for row in iter_row(cur, 10):
            print(row)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [15]:
if __name__ == '__main__':
    connect()
    create_tables()
    get_all_tweets("srcaballero99") #pass in the username of the account you want to insert
    get_all_tweets("Star_Lord987")
    get_all_tweets("Rush2Loot")
    simple_query_1()
    simple_query_2()
    join_query_1()
    aggregate_query_1()
    aggregate_query_2()
    search_query_1()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.1 on x86_64-apple-darwin, compiled by Apple LLVM version 6.0 (clang-600.0.54) (based on LLVM 3.5svn), 64-bit',)
Database connection closed.
getting tweets before 987094350722191359
...108 tweets downloaded so far
getting tweets before 1233178930330374143
...392 tweets downloaded so far
getting tweets before 1185190063975665665
...580 tweets downloaded so far
getting tweets before 1149644009029029891
...752 tweets downloaded so far
getting tweets before 1064253782823968768
...936 tweets downloaded so far
getting tweets before 995449260232626175
...1127 tweets downloaded so far
getting tweets before 935080812978065408
...1325 tweets downloaded so far
getting tweets before 785558030926680064
...1344 tweets downloaded so far
getting tweets before 598532734101954559
...1344 tweets downloaded so far
getting tweets before 1060464469959155712
...2 tweets downloaded so far
('("2020-01-20 12:07:25","AI Career P